# Molecular time evolution resource counts

Computes the resources to do Trotterized time evolution, a subroutine used in quantum Krylov and quantum phase estimation, of molecular Hamiltonians on IBM Quantum computers.

## Setup

In [1]:
import os
import datetime

import matplotlib.pyplot as plt
import numpy as np
import pickle
import scipy

import openfermion as of
from openfermionpyscf import run_pyscf
from openfermion.chem import geometry_from_pubchem, MolecularData

import qiskit
import qiskit.qasm3
import qiskit_ibm_runtime
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.synthesis import LieTrotter, SuzukiTrotter

import convert
import modified_hubbard
import openfermion_helper

from typing import Literal

In [33]:
# Parameters.
hamiltonian_name: str = "one-water-phosphate-ducc"

# For Hubbard model only.
xdim: int = 8  # Only for Hubbard, the x-dimension of the Hubbard model.
ydim: int = 8  # Only for Hubbard, the y-dimension of the Hubbard model.

threshold: float = 0.01  # Remove terms in Hamiltonian whose coefficients are smaller than this value in magnitude.

ibm_computer: str = "ibm_fez"  # IBM computer to compile to.
use_tket: bool = False  # Also run circuits through the TKET compiler if True.

save_circuit_counts: bool = True  # Option to save circuit counts (number of gates).
save_circuits: bool = True  # Option to save circuits (as QASM3 text files which can be loaded with qiskit.qasm3.load(filename)).
compute_shots: bool = False  # Also compute the number of shots required to measure the Hamiltonian to various accuracies, assuming the k-commuting method. Requires computing the ground state, only feasible for small Hamiltonians.

mapping: Literal["JW", "BK", "BKT"] = "BK" # choose between Jordan-Wigner, Bravyi-Kitaev, or Bravyi-Kitaev tree QubitOperator Mapping

In [34]:
# Parse parameters.
hamiltonian_options = ["one-water-phosphate", "one-water-phosphate-ducc", "water", "hubbard", "modified-hubbard"]
if hamiltonian_name not in hamiltonian_options:
    raise ValueError(
        f"Unknown Hamiltonian option. Options are {hamiltonian_options}. "
        "Custom Hamiltonian can be defined in the notebook."
    )

In [35]:
# Directory for saving.
if save_circuit_counts or save_circuits:
    time_key = datetime.datetime.now().strftime("%m_%d_%Y_%H:%M:%S")  # For saving results.
    save_directory = f"{hamiltonian_name}_"
    if hamiltonian_name in ("hubbard", "modified-hubbard"):
        save_directory += f"xdim_{xdim}_ydim_{ydim}_"
    save_directory += f"threshold_{threshold}_{time_key}"

    os.mkdir(save_directory)

## Get Hamiltonian

In [36]:
if hamiltonian_name == "one-water-phosphate-ducc":
    hamiltonian = of.utils.load_operator(file_name="owp_631gd_22_ducc.data", data_directory=".")

elif hamiltonian_name == "one-water-phosphate":
    geometry = [
        ("P", (-1.034220, -0.234256,0.672434)),
        ("O", (-1.004065, 0.890081, -0.334695)),
        ("O", (-0.003166, -1.329504, 0.557597)),
        ("O", (-2.065823, -0.232403, 1.765329)),
        ("H", (0.881055, 0.866924, -1.063283)),
        ("O", (1.748944, 0.417505, -1.047631)),
        ("H", (1.477276, -0.378346, -0.549750)),
    ]
    basis = "sto-3g"
    multiplicity = 1
    charge = 1

    hamiltonian_name = MolecularData(geometry, basis, multiplicity, charge)
    mol = run_pyscf(hamiltonian_name, run_mp2=True, run_cisd=False, run_ccsd=False, run_fci=False)
    mol.save()

    hamiltonian = MolecularData(filename=hamiltonian_name.filename)
    hamiltonian = hamiltonian.get_molecular_hamiltonian()
    hamiltonian = of.get_fermion_operator(hamiltonian)

elif hamiltonian_name == "water":
    geometry = geometry_from_pubchem("water")
    basis = "sto-3g"
    multiplicity = 1
    charge = 0

    # Make molecule.
    hamiltonian_name = MolecularData(geometry, basis, multiplicity, charge)

    mol = run_pyscf(hamiltonian_name, run_mp2=True, run_cisd=True, run_ccsd=True, run_fci=True)
    mol.save()

    hamiltonian = MolecularData(filename=hamiltonian_name.filename)
    hamiltonian = hamiltonian.get_molecular_hamiltonian()
    hamiltonian = of.get_fermion_operator(hamiltonian)

elif hamiltonian_name == "hubbard":
    hamiltonian = of.hamiltonians.fermi_hubbard(xdim, ydim, 1.0, 1.0)

elif hamiltonian_name == "modified-hubbard":
    hamiltonian = modified_hubbard.modified_fermi_hubbard_default_paramaters(xdim, ydim)

In [37]:
print(f"Fermionic Hamiltonian has {len(hamiltonian.terms)} term(s).")

Fermionic Hamiltonian has 341939 term(s).


In [38]:
hamiltonian.compress(abs_tol=threshold)
print(f"Compressed Fermionic Hamiltonian has {len(hamiltonian.terms)} term(s).")

Compressed Fermionic Hamiltonian has 25895 term(s).


In [39]:
if mapping == "JW":
    hamiltonian_openfermion = of.jordan_wigner(hamiltonian)
elif mapping == "BK":
    hamiltonian_openfermion = of.bravyi_kitaev(hamiltonian)
elif mapping == "BKT":
    hamiltonian_openfermion = of.bravyi_kitaev_tree(hamiltonian)
else:
    raise ValueError(f"{mapping} mapping not recorgnized")

nqubits = openfermion_helper.get_num_qubits(hamiltonian_openfermion)
nterms = len(hamiltonian_openfermion.terms)

print(f"Qubit Hamiltonian acts on {nqubits} qubit(s) and has {nterms} term(s).")

Qubit Hamiltonian acts on 44 qubit(s) and has 43519 term(s).


In [9]:
def average_pauli_weight(qubit_op: of.QubitOperator, weighted: bool = False, weight_func=abs) -> float:
    """
    Compute the average Pauli weight of a QubitOperator.
    
    Args:
        qubit_op: OpenFermion QubitOperator (sum of Pauli strings).
        weighted: If True, weight by weight_func(coeff) for each term.
        weight_func: Function applied to the complex coefficient to get a nonnegative weight (default abs).

    Returns:
        Average weight (float). Identity-only operators contribute weight 0.
    """
    items = list(qubit_op.terms.items())  # (term, coeff)
    if not items:
        return 0.0

    if not weighted:
        return sum(len(term) for term, _ in items) / len(items)

    # Coefficient-weighted average
    num = 0.0
    den = 0.0
    for term, coeff in items:
        w = len(term)                 # number of non-identity Paulis in the string
        s = float(weight_func(coeff)) # e.g., |coeff|
        num += w * s
        den += s
    return num / den if den else 0.0

non_weighted = average_pauli_weight(qubit_op=hamiltonian_openfermion, weighted=False)
weighted = average_pauli_weight(qubit_op=hamiltonian_openfermion, weighted=True)

print(f"Average Pauli weight: \n {non_weighted=} \n {weighted=}")

Average Pauli weight: 
 non_weighted=10.17135044463338 
 weighted=5.843882727306126


In [106]:
hamiltonian = openfermion_helper.preprocess_hamiltonian(
    hamiltonian_openfermion,
    drop_term_if=[lambda term: term == ()],
    verbose=True,
)  # Drop identity and convert to Cirq PauliSum.

nterms = len(hamiltonian)
nqubits = len(hamiltonian.qubits)

print(f"\n\nPre-processed qubit Hamiltonian acts on {nqubits} qubit(s) and has {nterms} term(s).")

'), (3, 'Y'), (4, 'Y'), (5, 'X'), (7, 'Z'), (11, 'Z'), (13, 'Y'), (15, 'Y'), (23, 'Y'))Status: On term 3215 = ((0, 'X'), (1, 'X'), (3, 'Y'), (4, 'Y'), (5, 'X'), (26, 'Z'))Status: On term 4228 = ((0, 'Z'), (1, 'X'), (3, 'Y'), (4, 'Z'), (5, 'X'), (7, 'Z'), (8, 'Z'), (9, 'X'), (11, 'X'), (15, 'Y'), (16, 'Z'), (17, 'X'), (19, 'X'), (23, 'X'))Status: On term 5072 = ((1, 'Y'), (3, 'Y'), (5, 'Z'), (6, 'Z'), (17, 'Y'), (19, 'Y'), (20, 'Z'), (21, 'X'))Status: On term 5866 = ((1, 'Z'), (2, 'Y'), (3, 'Y'), (5, 'Z'), (6, 'Z'), (24, 'X'), (25, 'Z'))Status: On term 6566 = ((1, 'Z'), (2, 'X'), (3, 'Y'), (5, 'Z'), (6, 'Y'), (30, 'Z'))Status: On term 7232 = ((1, 'Z'), (2, 'Z'), (3, 'X'), (7, 'Y'), (9, 'Z'), (10, 'Z'), (11, 'X'), (19, 'Z'), (21, 'Z'), (22, 'X'), (23, 'Y'), (27, 'Z'), (29, 'Z'), (30, 'X'))Status: On term 7853 = ((3, 'Z'), (4, 'Y'), (5, 'X'), (11, 'Z'), (13, 'Z'), (14, 'Y'), (15, 'Y'), (23, 'Y'))Status: On term 8405 = ((4, 'Z'), (7, 'Z'), (11, 'Z'), (13, 'Y'), (15, 'Y'), (16, 'Z'), (17, '

## Gate counts for a first order Trotter step

### (1) Crude estimate

In [23]:
# """Crude estimate."""
# num_cnots_crude: int = 0
# for term in hamiltonian:
#     num_cnots_crude += 2 ** (len(term.qubits) - 1)

# num_cnots_crude

### (2) Grouping + CNOT ladder

In [24]:
# """Estimate using grouping + diagonaliztion + exp(Z...Z) "ladder"."""
# import kcommute


# groups = kcommute.get_si_sets(hamiltonian, k=nqubits)

# num_cnots: int = 0
# for group in groups:
#     num_cnots += nqubits ** 2  # It takes O(n^2) Clifford gates to diagonalize all terms in this group [https://arxiv.org/abs/quant-ph/0406196].
#     for term in group:
#         num_cnots += 2 * len(term.qubits)  # Using 2w CNOTs in a "ladder" and one exp(Z) gate on the bottom qubit. See https://arxiv.org/abs/2408.08265v3 Fig. 3.
#     num_cnots += nqubits ** 2  # Rotating back to the Z basis (undoing the diagonal unitary).

# num_cnots

### (3) Qiskit's `PauliHedral` method

In [107]:
H = convert.cirq_pauli_sum_to_qiskit_pauli_op(hamiltonian)

In [108]:
# Following https://qiskit-community.github.io/qiskit-algorithms/tutorials/13_trotterQRTE.html.
order: int = 1
cx_structure = "chain"  # "fountain"
trotter_step = PauliEvolutionGate(H, time=1, synthesis=LieTrotter(cx_structure=cx_structure) if order == 1 else SuzukiTrotter(order, cx_structure=cx_structure))

circuit = qiskit.QuantumCircuit(H.num_qubits)
circuit.append(trotter_step, range(H.num_qubits));

circuit = circuit.decompose(reps=2)
circuit = qiskit.transpile(
    circuit,
    optimization_level=0,
    basis_gates=["u3", "cx"]
)

print(
    f"""
Depth: {circuit.depth()}
Gates: {", ".join([f"{k.upper()}: {v}" for k, v in circuit.count_ops().items()])}
"""
)
if save_circuit_counts:
    with open(f"{save_directory}/trotter_cx_u3_all_to_all_connectivity.pkl", "wb") as file:
        pickle.dump(circuit.count_ops(), file)

if save_circuits:
    with open(f"{save_directory}/trotter_cx_u3_all_to_all_connectivity.qasm3", "w") as file:
        qiskit.qasm3.dump(circuit, file)


Depth: 972814
Gates: U3: 1060790, CX: 798258


## Optimize the circuit

In [25]:
compiled = qiskit.transpile(
    circuit,
    optimization_level=3,
    basis_gates=["u3", "cx"]
)
print(
    f"""
Depth: {compiled.depth()}
Gates: {", ".join([f"{k.upper()}: {v}" for k, v in compiled.count_ops().items()])}
"""
)
if save_circuit_counts:
    with open(f"{save_directory}/trotter_optimized_cx_u3_all_to_all_connectivity.pkl", "wb") as file:
        pickle.dump(compiled.count_ops(), file)

if save_circuits:
    with open(f"{save_directory}/trotter_optimized_cx_u3_all_to_all_connectivity.qasm3", "w") as file:
        qiskit.qasm3.dump(compiled, file)


Depth: 775271
Gates: CX: 690102, U3: 193476


## Compile to an IBM quantum computer

In [ ]:
computer = qiskit_ibm_runtime.QiskitRuntimeService().backend(ibm_computer)  # Assumes a saved account.

compiled_to_computer = qiskit.transpile(
    compiled,
    backend=computer,
    optimization_level=3,
)
print(
    f"""
Depth: {compiled_to_computer.depth()}
Gates: {", ".join([f"{k.upper()}: {v}" for k, v in compiled_to_computer.count_ops().items()])}
"""
)
if save_circuit_counts:
    with open(f"{save_directory}/trotter_optimized_{ibm_computer}.pkl", "wb") as file:
        pickle.dump(compiled_to_computer.count_ops(), file)

if save_circuits:
    with open(f"{save_directory}/trotter_optimized_{ibm_computer}.qasm3", "w") as file:
        qiskit.qasm3.dump(compiled_to_computer, file)

### Using Quantinuum's TKET compiler

In [ ]:
if use_tket:
    from pytket.extensions.qiskit import IBMQBackend, tk_to_qiskit, qiskit_to_tk

    
    ibmbackend = IBMQBackend(backend_name=ibm_computer)  # Assumes a saved account. 

    tket_compiled = ibmbackend.get_compiled_circuit(
        circuit=qiskit_to_tk(circuit),
        optimisation_level=3,
    )
    compiled_tket = tk_to_qiskit(tket_compiled)

    print(
        f"""
    Depth: {compiled_tket.depth()}
    Gates: {", ".join([f"{k.upper()}: {v}" for k, v in compiled_tket.count_ops().items()])}
    """
    )
    if save_circuit_counts:
        with open(f"{save_directory}/trotter_optimized_tket_{ibm_computer}.pkl", "wb") as file:
            pickle.dump(compiled_tket.count_ops(), file)

    if save_circuits:
        with open(f"{save_directory}/trotter_optimized_tket_{ibm_computer}.qasm3", "w") as file:
            qiskit.qasm3.dump(compiled_tket, file)

## Compute the number of Trotter steps needed for chemical accuracy

The Trotter error $\epsilon$ decreases linearly in the number of Trotter stops $r$. Using error bounds for $\epsilon$ from https://arxiv.org/abs/1912.08854 and chemical accuracy $\epsilon^* = 10^{-3}$ Ha, we estimate the number of Trotter steps needed for chemical accuracy as $r = \epsilon / \epsilon^*$.

In [ ]:
error = np.sum(np.abs(H.coeffs))  # Loose error bound from https://arxiv.org/abs/1912.08854.

epsilon: float = 0.001  # mHa

nsteps = round(error / epsilon)
nsteps

## Compute number of shots

We compute the number of shots needed to compute one energy (cost function) $\langle \psi | H | \psi \rangle$ to accuracy $\epsilon$, assuming an (approximate) ground state $|\psi\rangle$.

In [21]:
if compute_shots:
    import kcommute


    eval, evec = scipy.sparse.linalg.eigsh(
        of.linalg.get_sparse_operator(hamiltonian_openfermion),
        k=1,
        which="SA"
    )
    evec = evec.flatten()

    epsilons = [0.1, 0.01, 0.001]
    kvals = [1, 2, nqubits // 2, nqubits]

    all_shots = []
    for k in kvals:
        groups = kcommute.get_si_sets(hamiltonian, k=k)
        groups_of = convert.to_groups_of(groups)
        base_shots = kcommute.compute_shots(groups_of, evec, epsilon=1)
        shots = [base_shots / epsilon ** 2 for epsilon in epsilons]
        all_shots.append(shots)
        print(all_shots)
    
    plt.rcParams.update({"font.family": "serif", "font.size": 12})


    for kval, shots in zip(kvals, all_shots):
        plt.loglog(epsilons, shots, "--o", alpha=0.75, mec="black", label=f"$k = {kval}$")

    plt.legend()
    plt.xlabel("Accuracy $\epsilon$")
    plt.ylabel("Shots $N$")
    plt.title(f"$k$-commuting shot counts w.r.t. ground state\n for {mol.name} ({nqubits} qubits, {nterms} Paulis)");